## ERA data processing 
    install Anaconda, use the following guide: https://www.anaconda.com/products/individual

    go to Anconda command line
    Create the following environment using the following command:
        conda create -n ERA5 ipython jupyterlab numpy scipy 
        conda install -c conda-forge earthengine-api -y
        conda activate ERA5
        jupyter lab 

In [1]:
import ee
import time
import math
import csv
import json
import os
import numpy as np
import datetime
from PIL import Image

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWtgzh76vlDtuiQ0Uq0Nga49FxTTFuCrQuB67XvMG7ST_v4HRVXh95UZsL8



Successfully saved authorization token.


In [40]:
def export_collection_to_drive(collection, num_images: int=0, image_names: list=[], crsdrive: str="" ,crstransformdrive: list=[],gdrive_folder: str="", scale: float=20, max_pixels: int=1e14,
                               data_type: str="float"):
    collection = ee.ImageCollection(collection)
    image_list = collection.toList(num_images)
    task_list = []

    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        name = image_names[i]
        # print(name)
        typed_images = {"double": image.toDouble(), "float": image.toFloat(), "byte": image.toByte(), "int": image.toInt()}
        export_task = ee.batch.Export.image.toDrive(image = typed_images[data_type],
                                                    description = name,
                                                    crs=crsdrive,
                                                    crsTransform=crstransformdrive,
                                                    folder = gdrive_folder,
                                                    fileNamePrefix = name,
                                                    region = image.geometry(),
                                                    scale = scale,
                                                    maxPixels = max_pixels)
        export_task.start()
        task_list.append(export_task) 

In [81]:
Alaska=ee.FeatureCollection('users/GangHong2/WP4/Met1km_Alaska')
Canada=ee.FeatureCollection('users/GangHong2/WP4/Met1km_Canada')
area=Canada

startDate=ee.Date('2021-07-01') 
endDate=ee.Date('2021-08-06')
numberOfDays = endDate.difference(startDate, 'days');  ## calcualte the number of days

## calculate temperature_2m max

In [82]:
parameter='temperature_2m'
## function for calculating the parameter
def calDaily_max(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .max() \  ## maximum of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))      

## run the calDaily function for ImageCollection
daily_temperature_max = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_max)
)

In [83]:
pro=daily_temperature_max.first().projection().getInfo()  ## get the projection information for the first file
output_names_max = ([name +'_'+parameter+'_max' for name in daily_temperature_max.toList(daily_temperature_max.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [84]:
export_task_temperature_2m_max =export_collection_to_drive(collection=daily_temperature_max,
                                                   num_images=daily_temperature_max.size().getInfo(),                                                
                                                   image_names = output_names_max,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,      ## 0.1 degree in the output                                           
                                                   gdrive_folder=parameter+'_max',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## calculate temperature_2m min

In [85]:
parameter='temperature_2m'
## function for calculating the parameter
def calDaily_min(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .min() \ ## minimum of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_temperature_min = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_min)
)

In [86]:
pro=daily_temperature_min.first().projection().getInfo()  ## get the projection information for the first file
output_names_min = ([name +'_'+parameter+'_min' for name in daily_temperature_min.toList(daily_temperature_min.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [87]:
export_task_temperature_2m_min =export_collection_to_drive(collection=daily_temperature_min,
                                                   num_images=daily_temperature_min.size().getInfo(),                                                
                                                   image_names = output_names_min,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,       ## 0.1 degree in the output                                          
                                                   gdrive_folder=parameter+'_min',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## calculate ave dewpoint_temperature_2m

In [88]:
parameter='dewpoint_temperature_2m'
## function for calculating the parameter
def calDaily_dewpoint_temperature_mean(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .mean() \ ## minimum of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_dewpoint_temperature_mean = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_dewpoint_temperature_mean)
)

In [89]:
pro=daily_dewpoint_temperature_mean.first().projection().getInfo()  ## get the projection information for the first file
output_names_mean = ([name +'_'+parameter+'_mean' for name in daily_dewpoint_temperature_mean.toList(daily_dewpoint_temperature_mean.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [90]:
export_task_dewpoint_temperature =export_collection_to_drive(collection=daily_dewpoint_temperature_mean,
                                                   num_images=daily_dewpoint_temperature_mean.size().getInfo(),                                                
                                                   image_names = output_names_mean,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,        ## 0.1 degree in the output                                         
                                                   gdrive_folder=parameter+'_mean',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## total_precipitation

In [91]:
parameter='total_precipitation_hourly'
## function for calculating the parameter
def calDaily_precipitation_sum(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .sum() \ ## total of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_precipitation_sum = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_precipitation_sum))

In [92]:
pro=daily_precipitation_sum.first().projection().getInfo()  ## get the projection information for the first file
output_names_sum = ([name +'_'+parameter+'_sum' for name in daily_precipitation_sum.toList(daily_precipitation_sum.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [93]:
export_task_precipitation =export_collection_to_drive(collection=daily_precipitation_sum,
                                                   num_images=daily_precipitation_sum.size().getInfo(),                                                
                                                   image_names = output_names_sum,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,      ## 0.1 degree in the output                                           
                                                   gdrive_folder=parameter+'_sum',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## total surface_solar_radiation_downwards

In [95]:
parameter='surface_solar_radiation_downwards'
## function for calculating the parameter
def calDaily_surface_solar_radiation_downwards_sum(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .sum() \ ## total of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_surface_solar_radiation_downwards_sum = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_surface_solar_radiation_downwards_sum))

In [96]:
pro=daily_surface_solar_radiation_downwards_sum.first().projection().getInfo()  ## get the projection information for the first file
output_names_sum = ([name +'_'+parameter+'_sum' for name in daily_surface_solar_radiation_downwards_sum.toList(daily_surface_solar_radiation_downwards_sum.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [97]:
export_task_surface_solar_radiation =export_collection_to_drive(collection=daily_surface_solar_radiation_downwards_sum,
                                                   num_images=daily_surface_solar_radiation_downwards_sum.size().getInfo(),                                                
                                                   image_names = output_names_sum,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,         ## 0.1 degree in the output                                        
                                                   gdrive_folder=parameter+'_sum',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## total surface_thermal_radiation_downwards

In [98]:
parameter='surface_thermal_radiation_downwards'
## function for calculating the parameter
def calDaily_surface_thermal_radiation_downwards_sum(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select(parameter) \
        .filterDate(start, end) \
        .sum() \ ## total of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_surface_thermal_radiation_downwards_sum = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_surface_thermal_radiation_downwards_sum))

In [99]:
pro=daily_surface_thermal_radiation_downwards_sum.first().projection().getInfo()  ## get the projection information for the first file
output_names_sum = ([name +'_'+parameter+'_sum' for name in daily_surface_thermal_radiation_downwards_sum.toList(daily_surface_thermal_radiation_downwards_sum.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [100]:
export_task_surface_thermal_radiation =export_collection_to_drive(collection=daily_surface_thermal_radiation_downwards_sum,
                                                   num_images=daily_surface_thermal_radiation_downwards_sum.size().getInfo(),                                                
                                                   image_names = output_names_sum,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,       ## 0.1 degree in the output                                          
                                                   gdrive_folder=parameter+'_sum',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## claculate wind component 

In [120]:
def cal_wind(image):
    wind = image.expression(
    '(u**2+v**2)**0.5', {
      'u': image.select('u_component_of_wind_10m'),
      'v': image.select('v_component_of_wind_10m'),
    }).rename('wind');
  
    time = image.get('system:time_start');
    return wind.set('date', ee.Date(time).format('YYYY-MM-dd')).set('system:time_start',time);
  

In [125]:
wind_dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('u_component_of_wind_10m','v_component_of_wind_10m')\
            .filter(ee.Filter.date(startDate, endDate))\
            .map(lambda image: ee.Image(image).clip(area));

component_wind = wind_dataset.map(cal_wind)

In [122]:
# component_wind= ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('u_component_of_wind_10m','v_component_of_wind_10m')\
#             .filter(ee.Filter.date(startDate, endDate))\
#             .map(lambda image: ee.Image(image).clip(area))\
#             .map(cal_wind)
# # component_wind = wind_dataset.map(component_wind)

In [126]:
parameter='wind'
## function for calculating the parameter
def calDaily_wind_mean(dayOffset):
    start = startDate.advance(dayOffset, 'days');
    end = start.advance(1, 'days');
    return ee.ImageCollection(component_wind).select(parameter) \
        .filterDate(start, end) \
        .mean() \ ## sum of the hourly data within the day
        .clip(area.geometry())\
        .set('system:time_start', start.millis()) \
        .set('date', ee.Date(start).format('YYYY-MM-dd'))           

## run the calDaily function for ImageCollection
daily_wind_mean = ee.ImageCollection(
  ee.List.sequence(0, numberOfDays.subtract(1))
    .map(calDaily_wind_mean))

In [127]:
pro=daily_wind_mean.first().projection().getInfo()  ## get the projection information for the first file
output_names_mean = ([name +'_'+parameter+'_mean' for name in daily_wind_mean.toList(daily_wind_mean.size()).map(lambda image: ee.Image(image).get('date')).getInfo()]) ## prepare the output name, format 'date+ parameter'

In [128]:
export_task_wind =export_collection_to_drive(collection=daily_wind_mean,
                                                   num_images=daily_wind_mean.size().getInfo(),                                                
                                                   image_names = output_names_mean,
                                                   crsdrive=pro['crs'],
                                                   crstransformdrive=pro['transform'],
                                                   scale=11132,              ## 0.1 degree in the output                               
                                                   gdrive_folder=parameter+'_mean',   ## use the parameter name to create a folder in Google Drive                                              
                                                   data_type='float',
                                                   max_pixels=1e13)

## convert tiff file to raw- download tiff files from Google Drive, the code below can convert all tif files in one directory to raw files in a new directry called raw

In [45]:
file_dir='E:\\temperature_2m\\'
out_dir='E:\\temperature_2m\\raw\\'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [46]:
for file in os.listdir(file_dir):
    if file.endswith('.tif'):      
        prefix_file=file[:-4]  ## get the file name without extension
        input_file=os.path.join(file_dir, file) ## get the full path of tif file
        output_file = os.path.join(out_dir, prefix_file+".raw")       ## get the full path of raw file
        im = Image.open(input_file)
        imarray = np.array(im)   ## conver to numpy array
        imarray.astype('float32').tofile(output_file)